In [4]:
import glob

import numpy as np
import pandas as pd
from scipy import signal

import librosa
import librosa.display as DSP
import librosa as lr
from tempfile import mktemp
# import imagehash
from PIL import Image

import pylab

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

import os
import wave
import time
import pickle
import warnings
import numpy as np
from sklearn import preprocessing
from scipy.io.wavfile import read
import python_speech_features as mfcc
from sklearn.mixture import GaussianMixture 
from sklearn.metrics import f1_score
import joblib
import pickle

In [5]:
def calculate_delta(array):
   
    rows, cols = array.shape
    # print(rows)
    # print(cols)
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
                first =0
            else:
                first = i-j
            if i+j > rows-1:
                second = rows-1
            else:
                second = i+j 
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

In [6]:
def extract_features(audio,rate):
       
    mfcc_feature = mfcc.mfcc(audio, rate, 0.025, 0.01, 20, nfft = 2205, appendEnergy = True)    
    mfcc_feature = preprocessing.scale(mfcc_feature)

    print(mfcc_feature)
    delta = calculate_delta(mfcc_feature)
    combined = np.hstack((mfcc_feature, delta)) 
    return combined



In [7]:
X_test = []
y_test = []

lstFolders = ["Close", "Open"]
for lst in lstFolders:
    listFiles = glob.glob(r"Speech data\{}\*".format(lst))
    features = np.asarray(())
    finishCounter = len(listFiles)
    counter = 1
    for file in listFiles:
        try:
        
            audio, sr = librosa.load(file)
            
            vector = extract_features(audio, sr)
            
            if features.size == 0:
                features = vector
            else:
                features = np.vstack((features, vector))
                
            X_test.append(vector)
            y_test.append(lstFolders.index(lst))
                
            counter += 1
            if(counter == finishCounter):
                gmm = GaussianMixture(n_components = 6, max_iter = 200, covariance_type='diag', n_init = 3)
                gmm.fit(features)
                
                # dumping the trained gaussian model
                picklefile = lst + ".sav"
                
                pickle.dump(gmm, open(picklefile, 'wb'))
        except:
            continue
      

[[-1.58759198 -2.11859181 -0.55587727 ...  0.2835796   0.69870693
   0.89359635]
 [-1.57192952 -2.19885776 -0.51135725 ...  0.38636     0.14943775
   0.55473575]
 [-1.57600174 -1.9369592  -0.11141834 ...  0.45523145  0.15454286
   0.14653109]
 ...
 [-0.59965214  0.27811814  0.25664349 ...  0.24205577 -0.52607069
   1.16931801]
 [-0.64416612  0.26298291  0.19323956 ...  0.28842159 -0.61949019
   0.03924237]
 [-0.67118444  0.24859818  0.08223806 ...  0.51957195 -1.32423481
  -0.8558293 ]]
[[-1.51813295 -2.20211643 -0.43188292 ...  0.17373588  0.72979213
   0.73679372]
 [-1.5031323  -2.28642824 -0.38578215 ...  0.25151241  0.21141574
   0.45323667]
 [-1.50703246 -2.01132851  0.0283572  ...  0.30362919  0.21623372
   0.11165282]
 ...
 [-0.55914018  0.11690023 -0.45314969 ... -0.85515833 -0.32108271
  -0.48871935]
 [-0.52353281  0.1800163  -0.40302029 ...  0.34522211  0.0903858
   0.27461077]
 [-0.60441201  0.22213024 -0.20403393 ...  1.42567395 -0.08685085
   0.63207749]]
[[-1.53076118 -2.

In [8]:
gmm_files = [ i + '.sav' for i in ["Close", "Open"]]

models    = [pickle.load(open(fname, 'rb')) for fname in gmm_files]


In [9]:
final_y = []
y = []
for i in X_test:
    log_likelihood = np.zeros(len(models)) 
    for j in range(len(models)):
        gmm = models[j] 
        scores = np.array(gmm.score(i))
        log_likelihood[j] = scores.sum()
    y.append(log_likelihood)

    winner = np.argmax(log_likelihood)
    print("\t detected as -",models[winner])
    final_y.append(winner)


	 detected as - GaussianMixture(covariance_type='diag', max_iter=200, n_components=6, n_init=3)
	 detected as - GaussianMixture(covariance_type='diag', max_iter=200, n_components=6, n_init=3)
	 detected as - GaussianMixture(covariance_type='diag', max_iter=200, n_components=6, n_init=3)
	 detected as - GaussianMixture(covariance_type='diag', max_iter=200, n_components=6, n_init=3)
	 detected as - GaussianMixture(covariance_type='diag', max_iter=200, n_components=6, n_init=3)
	 detected as - GaussianMixture(covariance_type='diag', max_iter=200, n_components=6, n_init=3)
	 detected as - GaussianMixture(covariance_type='diag', max_iter=200, n_components=6, n_init=3)
	 detected as - GaussianMixture(covariance_type='diag', max_iter=200, n_components=6, n_init=3)
	 detected as - GaussianMixture(covariance_type='diag', max_iter=200, n_components=6, n_init=3)
	 detected as - GaussianMixture(covariance_type='diag', max_iter=200, n_components=6, n_init=3)
	 detected as - GaussianMixture(covarian

In [10]:
f1_score(final_y, y_test, average='micro')

0.9074074074074074

In [11]:
file = r'C:\Users\DELL\Desktop\simple-recorder3\Training Data word\Open_book\rahma_open_book6 (4).wav'
audio, sr = librosa.load(file)
vector = extract_features(audio, sr)
og_likelihood = np.zeros(len(models)) 
for i in range(len(models)):
    gmm    = models[i] 
    scores = np.array(gmm.score(vector))
    log_likelihood[i] = scores.sum()
y.append(log_likelihood)

winner = np.argmax(log_likelihood)
print(winner)

g:\Dexter\anacondaaaaaaaaaaa\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\DELL\\Desktop\\simple-recorder3\\Training Data word\\Open_book\\rahma_open_book6 (4).wav'